In [9]:
from langchain_groq import ChatGroq
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from IPython.display import Image, display

In [10]:
groq_api_key = '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'
llm = ChatGroq(groq_api_key=groq_api_key, model_name="Gemma2-9b-It")

In [11]:
class State(TypedDict):
    messages: Annotated[list, add_messages]
    sentiment: str

# 1. Preprocessing Node
def preprocess(state: State) -> State:
    cleaned = state["messages"][-1].content.strip()
    state["messages"][-1].content = cleaned
    return state

# 2. Sentiment Analysis Node
def analyze_sentiment(state: State) -> State:
    msg = state["messages"][-1].content
    state["sentiment"] = "positive" if "good" in msg else "neutral"
    return state

# 3. Chatbot Node
def chatbot(state: State) -> State:
    return {"messages": llm.invoke(state['messages'])}

# 4. Logging Node
def logger(state: State) -> State:
    print(f"LOG: {state['messages'][-1].content}, Sentiment: {state.get('sentiment')}")
    return state

In [12]:
# Build the graph
builder = StateGraph(State)
builder.add_node("preprocess", preprocess)
builder.add_node("analyze_sentiment", analyze_sentiment)  # renamed
builder.add_node("chatbot", chatbot)
builder.add_node("logger", logger)

In [13]:
# Define flow
builder.add_edge(START, "preprocess")
builder.add_edge("preprocess", "analyze_sentiment")  # renamed
builder.add_edge("analyze_sentiment", "chatbot")     # renamed
builder.add_edge("chatbot", "logger")
builder.add_edge("logger", END)

In [14]:
graph = builder.compile()

In [15]:
# Draw the graph
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [16]:
# Invoke the graph
input_message = "This is a good test message."
final_state = graph.invoke({"messages": ("user", input_message)})

# Print the final state
print("Final State:", final_state)
print("Chatbot's response:", final_state['messages'][-1].content)
print("Detected Sentiment:", final_state['sentiment'])

input_message_2 = "This is a neutral message."
final_state_2 = graph.invoke({"messages": ("user", input_message_2)})
print("Final State 2:", final_state_2)
print("Chatbot's response 2:", final_state_2['messages'][-1].content)
print("Detected Sentiment 2:", final_state_2['sentiment'])

LOG: I agree! It's a clear and concise message. 

Is there anything else I can help you with? 😊  Perhaps you'd like to:

* **Ask me a question** about anything at all.
* **Give me a task** to complete, like writing a poem or summarizing a text.
* **Have a conversation** on a topic of your interest.


Let me know how I can be of service!
, Sentiment: positive
Final State: {'messages': [HumanMessage(content='This is a good test message.', additional_kwargs={}, response_metadata={}, id='44e431aa-b529-4149-8e47-42fc91ea6faa'), AIMessage(content="I agree! It's a clear and concise message. \n\nIs there anything else I can help you with? 😊  Perhaps you'd like to:\n\n* **Ask me a question** about anything at all.\n* **Give me a task** to complete, like writing a poem or summarizing a text.\n* **Have a conversation** on a topic of your interest.\n\n\nLet me know how I can be of service!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 94, 'prompt_tokens': 16, 't